In [57]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))    

### *Import frameworks and libraries*

In [58]:
from matplotlib.pyplot import imread,imshow
import matplotlib.pyplot as plt
from skimage.transform import resize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist
from tensorflow import image
import cv2
import glob
%matplotlib inline
print('Ready, set, go....')

In [59]:
# Import dataset into the dataframes
df = pd.read_csv('../input/digit-recognizer/train.csv')
df_test = pd.read_csv('../input/digit-recognizer/test.csv')

In [60]:
df.shape, df_test.shape

In [61]:
df.head()

In [62]:
x_test = df_test

In [63]:
# Split the training data into x train and y train and reshape the data x train and y train into 4 dimension
x_train_flattened = df.drop('label', axis = 1)
y_train = df['label']
x_train = x_train_flattened.values.reshape(42000,28,28,1)
x_test = x_test.values.reshape(28000,28,28,1)

In [64]:
x_train.shape , x_test.shape

In [65]:
plt.matshow(x_train[20])

In [66]:
y_train[20]

In [67]:
x_train = x_train / 255
x_test = x_test / 255

### *The steps in modeling with Tensorflow are typically:**

1. Create or import a model
2. Compile a model
3. Fit the model
4. Evaluate the model
5. Tuning
6. Evaluate

### *Splitting training and validation data*

In [68]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.10, random_state = 42)

### *Build model, evaluate the model and then tuning* 

In [69]:
model = Sequential()
model.add(Conv2D(64, kernel_size = (7, 7), strides = (1, 1), activation = 'relu', input_shape = (28,28,1)))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Conv2D(128, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(256, kernel_size = (3, 3), activation = 'relu'))
model.add(Flatten())
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

In [70]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.summary()

In [71]:
history = model.fit(x_train, y_train, batch_size = 100, epochs = 20)

In [72]:
pd.DataFrame(history.history).plot(figsize = (12,8))

In [73]:
model.evaluate(x_val, y_val)

### *Import image*

In [74]:
vinh_num = []
vinh_num_resize = []
for filenames in glob.glob('../input/paint-nums/*.png'):
    print(filenames)
    num = cv2.imread(filenames)
    vinh_num.append(num)

In [75]:
for num in vinh_num:
    plt.matshow(num)

### *Convert the image color to GRAY scale and resize*

In [76]:
for num in vinh_num:
    num_gray = cv2.cvtColor(num, cv2.COLOR_BGR2GRAY)
    plt.matshow(num_gray)
    num_gray_scaled = cv2.resize(num_gray, [28, 28])
    vinh_num_resize.append(num_gray_scaled)

In [77]:
for num_gray_scaled in vinh_num_resize:
    plt.matshow(num_gray_scaled)

### *Invert the color of the images*

In [78]:
inverted_scaled_gray_num = []
for num_gray_scaled in vinh_num_resize:
    num_gray_scaled_inverted = cv2.bitwise_not(num_gray_scaled)
    inverted_scaled_gray_num.append(num_gray_scaled_inverted)

In [79]:
for num_gray_scaled_inverted in inverted_scaled_gray_num:
    plt.matshow(num_gray_scaled_inverted)

In [80]:
inverted_scaled_gray_num = np.array(inverted_scaled_gray_num)
inverted_scaled_gray_num = inverted_scaled_gray_num.reshape(10, 784)
inverted_scaled_gray_num.shape

In [81]:
df_vinh_num = pd.DataFrame(inverted_scaled_gray_num)
df_vinh_num.head()

In [82]:
df_vinh_num_test = df_vinh_num.values.reshape(10, 28, 28, 1)

In [83]:
plt.matshow(df_vinh_num_test[0])

In [84]:
rows = 2
columns = 5

fig = plt.figure(figsize=(16,8))

predictions = np.argmax(model.predict(df_vinh_num_test), axis = 1).astype('int32').reshape(rows*columns,)

for i in range(1,rows*columns+1):
    ax = plt.subplot(rows, columns, i)
    plt.imshow(df_vinh_num_test[i-1])
    predicted_values = "Predicted: " + str(predictions[i-1])
    ax.set_title(predicted_values)

fig.show()